In [53]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.iv import IV2SLS
import IV_estimation as IV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [55]:
# Copy the dataframe
df2 = data.copy()

In [56]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [57]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [58]:
y = df2['log_market_share']
x = df2[['const']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['HP', 'Range', 'Chargetime']]

In [59]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

In [60]:
model.first_stage

,Price
R-squared,0.8037
Partial R-squared,0.0534
Shea's R-squared,0.0534
Partial F-statistic,15.045
P-value (Partial F-stat),0.0018
Partial F-stat Distn,chi2(3)
==========================,============
const,-2.012e+04
,(-0.2862)
Segment_B,8.05e+04


In [61]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -0.5555
Estimator:                    IV-2SLS   Adj. R-squared:                -0.6496
No. Observations:                 334   F-statistic:                    309.79
Date:                Thu, Apr 25 2024   P-value (F-stat)                0.0000
Time:                        13:49:20   Distribution:                 chi2(19)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.2507     1.5999    -3.2820     0.0010     -8.3864     -2.1150
Segment_B     -0.7875     0.8882    -0.8867     0.3752     -2.5283      0.9532
Segment_C     -0.8217     1.2548    -0.6548     0.5126     -3.2812      1.6377
Segment_D     -2.9660     2.2816    -1.2999     0.1936     -7.4380      1.5059
Segment_E     -8.5447     3.6309    -2.3533     0.0186     -15.661     -1.4283
Segment_F     -13.945     5.6681    -2.4603     0.0139     -25.054     -2.8358
Segment_M     -4.6895     1.7889    -2.6215     0.0088     -8.1957     -1.1834
Segment_J     -1.8498     1.5578    -1.1875     0.2350     -4.9031      1.2034
Year_2014      0.7164     1.9820     0.3614     0.7178     -3.1682      4.6009
Year_2015     -0.5247     1.9581    -0.2680     0.7887     -4.3625      3.3131
Year_2016     -1.0208     1.6393    -0.6227     0.5335     -4.2338      2.1922
Year_2017     -1.3741     1.6208    -0.8478     0.3966     -4.5507      1.8026
Year_2018     -2.1959     1.6989    -1.2925     0.1962     -5.5258      1.1339
Year_2019     -2.5720     1.7510    -1.4689     0.1419     -6.0038      0.8598
Year_2020     -3.3571     1.6593    -2.0232     0.0431     -6.6092     -0.1050
Year_2021     -4.4595     1.6699    -2.6705     0.0076     -7.7324     -1.1865
Year_2022     -5.3581     1.7598    -3.0447     0.0023     -8.8073     -1.9089
Year_2023     -5.5117     1.6996    -3.2430     0.0012     -8.8428     -2.1806
China          1.2606     0.9738     1.2945     0.1955     -0.6481      3.1692
Price       1.641e-05   6.67e-06     2.4605     0.0139   3.339e-06   2.949e-05
==============================================================================

Endogenous: Price
Instruments: HP, Range, Chargetime
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Test

In [62]:
df1 = pd.read_csv('dataset.csv')
Nobs=df1['ID'].count()
df1['const']=np.ones((Nobs,1))
df1 = df1[df1['Market share'] != 0]
#df1['HP_instrument'] = 0
df1.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [63]:
models = df1['Model'].unique()
HP = df1['HP']
print(models)

['U5' 'U6' 'e-tron' 'e-tron GT' 'Q4 e-tron' 'Q8 e-tron' 'I3' 'i4' 'i5'
 'I7' 'iX' 'iX1' 'iX3' 'Atto 3' 'Dolphin' 'Han' 'Seal' 'Tang' 'Berlingo'
 'C4' 'ë-C4 X' 'e-SpaceTourer' 'Jumpy' 'Born' 'Spring' '3 Crossback' '500'
 'Ocean' 'Mustang Mach-E' 'E' 'e:Ny1' 'E-HS9' 'Ioniq' 'Ioniq 5' 'Ioniq 6'
 'Kona' 'e-JS4' 'I-Pace' 'Avenger' 'EV6' 'EV9' 'Niro' 'Soul' 'RZ' 'UX'
 'Euniq' 'Euniq5' 'Euniq6' 'MIFA 9' 'MX-30' 'EQA' 'EQB' 'EQC' 'EQE'
 'EQE SUV' 'EQS' 'EQS SUV' 'EQV' '4' '5' 'Marvel R' 'ZS' '3-dørs'
 'Cabriolet' 'EL6' 'ES8' 'ET5' 'ET7' 'Ariya' 'Leaf' 'Combo' 'Corsa'
 'Mokka' 'Zafira' '2008' '208' 'Expert' 'Rifter' 'Traveller' '2' 'Taycan'
 'Megane' 'Zoe' 'Mii' 'Citigo' 'Enyaq iV' 'Fortwo' 'Korando' 'Solterra'
 'Model 3' 'Model S' 'Model X' 'Model Y' 'bZ4X' 'Proace City Verso'
 'Proace Verso' 'Golf' 'ID.3' 'ID.4' 'ID.5' 'ID.7' 'ID.Buzz' 'up!' 'C40'
 'EX30' 'XC40' 'Free']


In [64]:
def HP_instrument1(df1):
    z = 0
    models = df1['Model'].unique()
    for model in models:
        for i in df1:
            if df1[model] == df1['Model'][i]:
                z = z
            else:
                z += df1['HP'][i]
    df1['HP_instrument'] = z            

In [65]:
def create_instrument(df1, instrument):
    df1[instrument+'_instrument'] = 0
    for index, row in df1.iterrows():
        current_model = row['Model']
        hp_sum_except_current = df1[df1['Model'] != current_model][instrument].sum()
        df1.at[index, instrument+'_instrument'] = hp_sum_except_current


In [66]:
create_instrument(df1, 'Chargetime')

In [67]:
df1

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,Chargetime_instrument
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0,11308
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0,11308
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0,11308
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0,11376
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,11325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,11382
1163,190,2021,0.014652,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,11326
1164,190,2022,0.033310,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,11326
1165,190,2023,0.031752,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,11326
